In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab as pl

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Получаем данные:

In [ ]:
df = pd.read_csv('/kaggle/input/indian-food-101/indian_food.csv')
df

* prep_time - время подготовки
* cook_time - время готовки

In [ ]:
df.describe()

# Какие есть диеты:

In [ ]:
df.diet.unique()

# Какие есть вкусы:

In [ ]:
df.flavor_profile.unique()

# Типы блюд:

In [ ]:
df.course.unique()

# Регионы:

In [ ]:
df.region.unique()

# Удаляем строки где есть плохие значения:

In [ ]:
df = df.drop(df[df.flavor_profile == '-1'].index)
df = df.drop(df[df.prep_time == -1].index)
df = df.drop(df[df.cook_time == -1].index)
df = df.drop(df[df.region == '-1'].index)
df = df.dropna()
df

# Выбираем данные:

In [ ]:
cdf = df[['diet', 'prep_time', 'cook_time', 'flavor_profile', 'course', 'region', 'name']]
cdf = pd.get_dummies(cdf, columns=['diet', 'flavor_profile', 'course'])
cdf

In [ ]:
viz = cdf[['prep_time', 'cook_time']]
viz.hist()
plt.show()

In [ ]:
plt.scatter(cdf.prep_time, cdf.cook_time, color='blue')
plt.xlabel('prep_time')
plt.ylabel('cook_time')
plt.show()

# Подготовка данных к обучению и тестированию модели:

In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

X = train.drop(['region', 'name'], axis=1)
y = train[['region', 'name']]

# Обучение модели:

In [ ]:
model.fit(X, y)

# Тренировка модели:

In [ ]:
from sklearn.metrics import r2_score

test_X = train.drop(['region', 'name'], axis=1)
test_y = train[['region', 'name']]
result = model.predict(test_X)

result

# Для примера создаю словарь с ключами колонками и значениями = 0:

In [ ]:
{ col:[0] for col in X.columns }

# Задаю параметры:

In [ ]:
example = {
    'prep_time': [17],
    'cook_time': [70],
    'diet_non vegetarian': [1],
    'diet_vegetarian': [0],
    'flavor_profile_bitter': [0],
    'flavor_profile_sour': [0],
    'flavor_profile_spicy': [1],
    'flavor_profile_sweet': [0],
    'course_dessert': [0],
    'course_main course': [0],
    'course_snack': [1],
    'course_starter': [0]
}

# Получаю предсказание:

In [ ]:
example_df = pd.DataFrame(example)
model.predict(example_df)

In [ ]:
model.predict_proba(example_df)

In [ ]:
model.classes_

In [ ]:
import pickle

In [ ]:
filename = 'indian_kitchen.sav'
pickle.dump(model, open(filename, 'wb'))